In [ ]:
!pip install openai>=1.0.0 geotext transformers GeoText
!pip install -q -U google-genai
!pip install langchain openai google-api-python-client langchain_community tools langchain_google_community langchain_openai

In [ ]:
!pip install --upgrade openai
!pip install gradio openpyxl

In [ ]:
from openai import OpenAI
import pandas as pd
import time
from datetime import datetime
import logging
import csv
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import re
from geotext import GeoText
import gradio as gr

In [ ]:
import os
from langchain import GoogleSearchAPIWrapper
from openai import OpenAI
from langchain_google_community import GoogleSearchAPIWrapper
from langchain_openai import ChatOpenAI
from langchain.agents import Tool, AgentExecutor, create_tool_calling_agent
from langchain.prompts import ChatPromptTemplate

In [ ]:
os.environ["IDA_LLM_API_KEY"]="your_key_here"
os.environ["GOOGLE_API_KEY"] = "your_key_here"
os.environ["GOOGLE_CSE_ID"] = "your_key_here"

In [ ]:
j = 0

In [ ]:
# Initialize your private LLM client (llama-3.3-70b-instruct via university server)
client = OpenAI(
    base_url="http://api.llm.apps.os.dcs.gla.ac.uk/v1",
    api_key=os.environ['IDA_LLM_API_KEY']
)

In [ ]:
def initialize_database():
    """Initialize the CSV database with all COLA framework columns"""
    if not os.path.exists('cola_database.csv'):
        columns = [
            'ID', 'Original_Query', 'Rewritten_Query', 'Selected_Topic_Intent', 'Selected_Answer_Type',
            'Linguist Analysis', 'Expert Analysis', 'User Analysis', 'In Favor', 'Against',
            'Final Judgement', 'After RAG Agent', 'Final Plan',
            'Processing_Time_Seconds', 'Processing_Time_Seconds_RAG','Timestamp', 'Status'
        ]
        df = pd.DataFrame(columns=columns)
        # Set proper data types to avoid warnings
        df = df.astype({
            'ID': 'int64',
            'Original_Query': 'string',
            'Rewritten_Query': 'string',
            'Selected_Topic_Intent': 'string',
            'Selected_Answer_Type': 'string',
            'Linguist Analysis': 'string',
            'Expert Analysis': 'string',
            'User Analysis': 'string',
            'In Favor': 'string',
            'Against': 'string',
            'Final Judgement': 'string',
            'After RAG Agent': 'string',
            'Final Plan': 'string',
            'Processing_Time_Seconds': 'float64',
            'Processing_Time_Seconds_RAG': 'float64',
            'Timestamp': 'string',
            'Status': 'string'
        })
        df.to_csv('cola_database.csv', index=False)
        print("Enhanced COLA database initialized")

In [ ]:
def add_new_query(query_id, query_text):
    """Add a new query to the database with pending status"""
    try:
        if os.path.exists('cola_database.csv'):
            df = pd.read_csv('cola_database.csv', dtype={
                'ID': 'int64',
                'Original_Query': 'string',
                'Rewritten_Query': 'string',
                'Selected_Topic_Intent': 'string',
                'Selected_Answer_Type': 'string',
                'Linguist Analysis': 'string',
                'Expert Analysis': 'string',
                'User Analysis': 'string',
                'In Favor': 'string',
                'Against': 'string',
                'Final Judgement': 'string',
                'After RAG Agent': 'string',
                'Final Plan': 'string',
                'Processing_Time_Seconds': 'float64',
                'Processing_Time_Seconds_RAG': 'float64',
                'Timestamp': 'string',
                'Status': 'string'
            })
        else:
            initialize_database()
            df = pd.read_csv('cola_database.csv', dtype={
                'ID': 'int64',
                'Original_Query': 'string',
                'Rewritten_Query': 'string',
                'Selected_Topic_Intent': 'string',
                'Selected_Answer_Type': 'string',
                'Linguist Analysis': 'string',
                'Expert Analysis': 'string',
                'User Analysis': 'string',
                'In Favor': 'string',
                'Against': 'string',
                'Final Judgement': 'string',
                'After RAG Agent': 'string',
                'Final Plan': 'string',
                'Processing_Time_Seconds': 'float64',
                'Processing_Time_Seconds_RAG': 'float64',
                'Timestamp': 'string',
                'Status': 'string'
            })

        # Add new query row
        new_row = {
            'ID': query_id,
            'Original_Query': query_text,
            'Rewritten_Query': '',
            'Selected_Topic_Intent': '',
            'Selected_Answer_Type': '',
            'Linguist Analysis': '',
            'Expert Analysis': '',
            'User Analysis': '',
            'In Favor': '',
            'Against': '',
            'Final Judgement': '',
            'After RAG Agent': '',
            'Final Plan': '',
            'Processing_Time_Seconds': None,
            'Processing_Time_Seconds_RAG': None,
            'Timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'Status': 'pending'
        }

        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
        df.to_csv('cola_database.csv', index=False)
        print(f"Added query {query_id} to enhanced COLA database")

    except Exception as e:
        print(f"Error adding query to database: {e}")


In [ ]:
def update_query_results(query_id, results_dict):
    """Update all COLA results for a specific query ID"""
    try:
        df = pd.read_csv('cola_database.csv', dtype={
            'ID': 'int64',
            'Original_Query': 'string',
            'Rewritten_Query': 'string',
            'Selected_Topic_Intent': 'string',
            'Selected_Answer_Type': 'string',
            'Linguist Analysis': 'string',
            'Expert Analysis': 'string',
            'User Analysis': 'string',
            'In Favor': 'string',
            'Against': 'string',
            'Final Judgement': 'string',
            'After RAG Agent': 'string',
            'Final Plan': 'string',
            'Processing_Time_Seconds': 'float64',
            'Processing_Time_Seconds_RAG': 'float64',
            'Timestamp': 'string',
            'Status': 'string'
        })

        # Find the row with the specific ID
        mask = df['ID'] == query_id
        if mask.any():
            # Update all columns with results - convert values to proper types
            for column, value in results_dict.items():
                if column in df.columns:
                    if column == 'Processing_Time_Seconds' and value is not None:
                        df.loc[mask, column] = float(value)
                    else:
                        df.loc[mask, column] = str(value) if value is not None else ''

            df.loc[mask, 'Status'] = 'completed'
            df.to_csv('cola_database.csv', index=False)
            print(f"Updated all results for query {query_id}")
        else:
            print(f"Query ID {query_id} not found in database")

    except Exception as e:
        print(f"Error updating results in database: {e}")

In [ ]:
def get_completion(prompt):
    max_retries = 100

    for i in range(max_retries):
        try:
          messages = [{"role": "user", "content": prompt}]
          response = client.chat.completions.create(
              model="llama-3.3-70b-instruct",
              messages=messages,
              temperature=0
            )
          return response.choices[0].message.content
        except Exception as e:  # Generic exception handling
            if i < max_retries - 1:
                time.sleep(2)
                logging.warning(f"Attempt {i+1} failed: {e}")
            else:
                logging.error(f'Max retries reached for prompt: {instruction}. Error: {e}')
                return "Error"



In [ ]:
def get_completion_with_role(role, instruction, content):
    max_retries = 100
    for i in range(max_retries):
      try:

        messages = [
            {"role": "system", "content": f"You are a {role}."},
            {"role": "user", "content": f"{instruction}\n{content}"}
        ]
        response = client.chat.completions.create(
            model="llama-3.3-70b-instruct",
            messages=messages,
            temperature=0
          )
        return response.choices[0].message.content

      except Exception as e:  # Generic exception handling
            if i < max_retries - 1:
                time.sleep(2)
                logging.warning(f"Attempt {i+1} failed: {e}")
            else:
                logging.error(f'Max retries reached for prompt: {instruction}. Error: {e}')
                return "Error"

In [ ]:
def generate_intent_options(original_query):
    """
    Generate 3 most likely DOMAIN/TOPIC interpretations for the original query.
    This disambiguates between completely different subjects, not just different angles.
    """
    prompt = f"""Query: "{original_query}"

        CRITICAL: I need you to identify if this query has AMBIGUOUS WORDS that could mean completely different things.

        Look for words that could be:
        - Programming language vs. place vs. other meanings (Java, Python, Ruby, etc.)
        - Company vs. fruit vs. other (Apple, Orange, etc.)
        - Person vs. place vs. concept (Tesla, Darwin, etc.)
        - Multiple different meanings entirely

        If you find ambiguous words, give me 3 DIFFERENT DOMAINS/SUBJECTS.
        If no ambiguous words, give me 3 different CONTEXTS for the same topic.

        WRONG (all same domain):
        - Java web development features
        - Java mobile app development
        - Java enterprise applications

        RIGHT (different domains):
        - Java (programming language)
        - Java (Indonesian island)
        - Java (coffee culture)

        Format: Topic (context)

        Respond with exactly 3 lines, no explanations:"""

    try:
        response = get_completion(prompt)

        # Clean and parse the response - be more aggressive about filtering
        lines = [line.strip() for line in response.strip().split('\n') if line.strip()]

        options = []
        # More comprehensive filtering for instruction text
        skip_phrases = [
            "here are", "results", "the query", "could refer", "examples",
            "format", "write only", "respond with", "provide", "interpretations",
            "different meanings", "ambiguous", "critical"
        ]

        for line in lines:
            # Skip lines that contain instruction-like phrases
            line_lower = line.lower()
            if any(phrase in line_lower for phrase in skip_phrases):
                continue

            # Remove numbering/bullets more aggressively
            cleaned_line = line
            import re
            cleaned_line = re.sub(r'^[0-9]+[\.\)\-\s]+', '', cleaned_line)
            cleaned_line = re.sub(r'^[\-\*\•]\s+', '', cleaned_line)

            # Only keep lines that look like actual topic options (should contain parentheses ideally)
            if cleaned_line and len(cleaned_line) > 3 and not any(phrase in cleaned_line.lower() for phrase in skip_phrases):
                options.append(cleaned_line)

        # Take first 3 options
        if len(options) >= 3:
            return options[:3]

        # If we don't get good results, create manual disambiguation for common terms
        query_lower = original_query.lower()

        # Check for common ambiguous terms
        if 'java' in query_lower:
            return [
                "Java (programming language)",
                "Java (Indonesian island)",
                "Java (coffee culture)"
            ]
        elif 'python' in query_lower:
            return [
                "Python (programming language)",
                "Python (snake species)",
                "Python (Monty Python comedy)"
            ]
        elif 'tesla' in query_lower:
            return [
                "Tesla (car company)",
                "Tesla (Nikola Tesla scientist)",
                "Tesla (band/music)"
            ]
        else:
            # Generic contextual fallback
            return [
                f"Technical/professional information about {original_query}",
                f"General educational information about {original_query}",
                f"Practical applications of {original_query}"
            ]

    except Exception as e:
        print(f"Error generating intent options: {e}")
        return [
            f"Technical information about {original_query}",
            f"General information about {original_query}",
            f"Practical guide for {original_query}"
        ]

In [ ]:
def process_selected_intent(selected_option, original_query, query_id, chatbot_history, state):
    """
    Process the user's selected intent and continue with COLA framework.
    """
    try:
        # Show confirmation of selection in simple text
        confirmation_msg = f"\n\n✅ **Selected Intent:** {selected_option}\n\n🤔 Processing through COLA framework..."

        confirmation_message = {"role": "assistant", "content": confirmation_msg}
        temp_history = chatbot_history + [confirmation_message]

        yield temp_history, state, ""

        # Now process with the selected intent as the working query
        answer = add_predictions_sequential(original_query, selected_option, query_id)

        # Replace confirmation with final result
        final_message = {"role": "assistant", "content": str(answer)}
        final_history = chatbot_history + [final_message]
        final_state = state + [final_message]

        yield final_history, final_state, ""

    except Exception as e:
        error_msg = f"\n\n❌ **Error processing selected intent:** {str(e)}"
        error_message = {"role": "assistant", "content": error_msg}
        error_history = chatbot_history + [error_message]
        error_state = state + [error_message]
        yield error_history, error_state, ""

In [ ]:
def show_intent_options(chatbot_history, state, options):
    """
    Display intent options to user for selection using simple text format.
    Returns updated chat history with the options.
    """
    # Use simple markdown formatting instead of HTML
    options_text = "🎯 **Please clarify your intent:**\n\n"
    options_text += "Select the option that best matches what you're looking for:\n\n"

    for i, option in enumerate(options, 1):
        options_text += f"**Option {i}:** {option}\n\n"

    options_text += "👇 **Click the corresponding Option button below the chat to proceed.**"

    # Add the options message to chat
    intent_message = {"role": "assistant", "content": options_text}
    updated_history = chatbot_history + [intent_message]
    updated_state = state + [intent_message]

    return updated_history, updated_state

In [ ]:
def generate_answer_type_options(query, selected_topic_intent):
    """
    Generate answer type options based on the query and selected topic intent.
    This helps clarify what kind of response the user is looking for.
    """
    prompt = f"""Query: "{query}"
        Selected Topic: "{selected_topic_intent}"

        The user wants to know about this topic. What type of answer would be most helpful?

        Generate 3 different ANSWER TYPE options that would be appropriate for this query:

        Consider these categories:
        - Informative (detailed explanation, facts, background information)
        - Practical Tips (actionable advice, how-to guidance, steps to follow)
        - Basic Overview (simple introduction, key points, beginner-friendly)
        - Expert Analysis (in-depth professional perspective, technical details)
        - Comparison/Evaluation (pros/cons, alternatives, recommendations)
        - Problem-Solving (solutions, troubleshooting, addressing specific issues)

        Format each option as: "Answer Type (brief description)"

        Examples:
        - Informative (comprehensive background and facts)
        - Practical Tips (step-by-step actionable guidance)
        - Basic Overview (simple introduction for beginners)

        Respond with exactly 3 lines, no explanations:"""

    try:
        response = get_completion(prompt)

        # Parse response similar to intent options
        lines = [line.strip() for line in response.strip().split('\n') if line.strip()]

        options = []
        skip_phrases = [
            "here are", "results", "the query", "could refer", "examples",
            "format", "write only", "respond with", "provide", "options",
            "different types", "answer type"
        ]

        for line in lines:
            line_lower = line.lower()
            if any(phrase in line_lower for phrase in skip_phrases):
                continue

            # Remove numbering/bullets
            import re
            cleaned_line = re.sub(r'^[0-9]+[\.\)\-\s]+', '', line)
            cleaned_line = re.sub(r'^[\-\*\•]\s+', '', cleaned_line)

            if cleaned_line and len(cleaned_line) > 3:
                options.append(cleaned_line)

        # Take first 3 options
        if len(options) >= 3:
            return options[:3]

        # Fallback options based on query analysis
        query_lower = query.lower()

        if any(word in query_lower for word in ['how to', 'steps', 'guide', 'tutorial']):
            return [
                "Practical Tips (step-by-step actionable guidance)",
                "Informative (detailed explanation and background)",
                "Basic Overview (simple introduction for beginners)"
            ]
        elif any(word in query_lower for word in ['what is', 'explain', 'about']):
            return [
                "Informative (comprehensive background and facts)",
                "Basic Overview (simple introduction for beginners)",
                "Expert Analysis (in-depth professional perspective)"
            ]
        else:
            # Generic fallback
            return [
                "Informative (comprehensive background and facts)",
                "Practical Tips (actionable advice and guidance)",
                "Basic Overview (simple introduction and key points)"
            ]

    except Exception as e:
        print(f"Error generating answer type options: {e}")
        return [
            "Informative (comprehensive background and facts)",
            "Practical Tips (actionable advice and guidance)",
            "Basic Overview (simple introduction and key points)"
        ]

In [ ]:
def show_answer_type_options(chatbot_history, state, options):
    """
    Display answer type options to user for selection.
    """
    options_text = "📝 **What type of answer would you like?**\n\n"
    options_text += "Select the format that best matches what you're looking for:\n\n"

    for i, option in enumerate(options, 1):
        options_text += f"**Type {i}:** {option}\n\n"

    options_text += "👇 **Click the corresponding Type button below the chat to proceed.**"

    # Add the options message to chat
    answer_type_message = {"role": "assistant", "content": options_text}
    updated_history = chatbot_history + [answer_type_message]
    updated_state = state + [answer_type_message]

    return updated_history, updated_state

In [ ]:
def process_selected_answer_type(selected_answer_type, selected_topic_intent, original_query, query_id, chatbot_history, state):
    """
    Process the user's selected answer type and continue with COLA framework.
    """
    try:
        # Show confirmation of selection
        confirmation_msg = f"✅ **Selected Topic:** {selected_topic_intent}\n"
        confirmation_msg += f"✅ **Selected Answer Type:** {selected_answer_type}\n\n"
        confirmation_msg += "🤔 Processing through COLA framework..."

        confirmation_message = {"role": "assistant", "content": confirmation_msg}
        temp_history = chatbot_history + [confirmation_message]

        yield temp_history, state, ""

        # Now process with both clarifications
        answer = add_predictions_sequential_enhanced(original_query, selected_topic_intent, selected_answer_type, query_id)

        # Replace confirmation with final result
        final_message = {"role": "assistant", "content": str(answer)}
        final_history = chatbot_history + [final_message]
        final_state = state + [final_message]

        yield final_history, final_state, ""

    except Exception as e:
        error_msg = f"\n\n❌ **Error processing selected answer type:** {str(e)}"
        error_message = {"role": "assistant", "content": error_msg}
        error_history = chatbot_history + [error_message]
        error_state = state + [error_message]
        yield error_history, error_state, ""


In [ ]:
def rewrite_query_with_dual_intent(original_query, selected_topic, selected_answer_type):
    """
    Rewrite the query using both the selected topic and desired answer type.
    This creates a comprehensive query that guides the COLA framework appropriately.
    """
    # Extract the main answer type from the selection
    answer_type_main = selected_answer_type.split('(')[0].strip()
    answer_type_description = selected_answer_type.split('(')[1].strip(')') if '(' in selected_answer_type else ""

    instruction = f"""You have an original user query, their selected topic/domain, and their desired answer type.

    Create a single, clear, and comprehensive query that:
    - Focuses specifically on the selected topic/domain: {selected_topic}
    - Indicates the type of response they want: {answer_type_main}
    - Preserves the user's original intent
    - Is suitable for expert analysis
    - Guides experts to provide the right format of response

    The rewritten query should naturally incorporate both the topic focus and the answer type preference.

    Examples:
    - Original: "tell me about java" + Topic: "Java (programming)" + Type: "Practical Tips"
      → "Provide practical tips and actionable guidance for learning and working with Java programming language"

    - Original: "what is bitcoin" + Topic: "Bitcoin (cryptocurrency)" + Type: "Basic Overview"
      → "Give a basic overview and introduction to Bitcoin cryptocurrency for beginners"

    Respond with ONLY the rewritten query. Do not include explanations or formatting."""

    prompt = f"""{instruction}

    Original Query: "{original_query}"
    Selected Topic/Domain: "{selected_topic}"
    Selected Answer Type: "{selected_answer_type}"

    Rewritten Query:"""

    try:
        response = get_completion(prompt)
        # Clean up the response
        cleaned_response = response.strip().strip('"').strip("'")

        # Remove common prefixes if they appear
        prefixes_to_remove = ["Rewrite:", "Rewritten Query:", "Query:", "Rewritten:"]
        for prefix in prefixes_to_remove:
            if cleaned_response.startswith(prefix):
                cleaned_response = cleaned_response[len(prefix):].strip()

        return cleaned_response if cleaned_response else f"{selected_topic} - {answer_type_main}"
    except Exception as e:
        print(f"Error in dual intent query rewriting: {e}")
        return f"{selected_topic} - {answer_type_main}"

In [ ]:
#GET ROLES
def get_roles(query):
    max_retries = 100
    prompt_roles = f"""Act as Recruiting Manager, considering the content of the query you are reading {query}, to what topic does
                    the question refer to and provide 3 expert roles to analyze how to solve the question.
                    These 3 roles should have different perspectives.

                    Return ONLY a Python list in this exact format:
                    ["topic name", "expert role 1", "expert role 2", "expert role 3"]

                    Do not include any other text, explanations, or formatting. Just the list."""
    for i in range(max_retries):
        try:
          messages = [{"role": "user", "content": prompt_roles}]
          response = client.chat.completions.create(
              model="llama-3.3-70b-instruct",
              messages=messages,
              temperature=0
            )
          return response.choices[0].message.content
        except Exception as e:  # Generic exception handling
            if i < max_retries - 1:
                time.sleep(2)
                logging.warning(f"Attempt {i+1} failed: {e}")
            else:
                logging.error(f'Max retries reached for prompt: {instruction}. Error: {e}')
                return "Error"

In [ ]:
def local_analysis_enhanced(query, topic, answer_type):
    """Enhanced local analysis with answer type consideration"""
    role = target_role_map.get("Local")

    # Extract the main answer type from the selection
    answer_type_main = answer_type.split('(')[0].strip()

    instruction = f"""You are a {role} with deep knowledge about {topic}.

    User Query: "{query}"
    Requested Answer Type: {answer_type}

    As a {role}, provide your professional analysis addressing this query.

    IMPORTANT: Format your response as {answer_type_main.upper()}:

    {get_answer_type_instructions(answer_type_main)}

    Your response should reflect the expertise and viewpoint that defines your role as a {role} while following the requested answer format."""

    return get_completion_with_role(role, instruction, query)

def expert_analysis_enhanced(query, topic, answer_type):
    """Enhanced expert analysis with answer type consideration"""
    role = target_role_map.get("Expert")
    answer_type_main = answer_type.split('(')[0].strip()

    instruction = f"""You are a {role} specializing in {topic}.

    User Query: "{query}"
    Requested Answer Type: {answer_type}

    Provide your professional expert analysis of this query.

    IMPORTANT: Format your response as {answer_type_main.upper()}:

    {get_answer_type_instructions(answer_type_main)}

    Your analysis should reflect the authority and comprehensive understanding that comes from being a recognized {role} in this domain."""

    return get_completion_with_role(role, instruction, query)

def user_analysis_enhanced(query, topic, answer_type):
    """Enhanced user analysis with answer type consideration"""
    role = target_role_map.get("User Analysis")
    answer_type_main = answer_type.split('(')[0].strip()

    instruction = f"""You are a {role} with expertise in {topic}.

    User Query: "{query}"
    Requested Answer Type: {answer_type}

    Analyze this query from your specialized perspective as a {role}.

    IMPORTANT: Format your response as {answer_type_main.upper()}:

    {get_answer_type_instructions(answer_type_main)}

    Your analysis should complement other expert perspectives while offering the distinct value that only a {role} can provide."""

    return get_completion_with_role(role, instruction, query)

In [ ]:
def get_answer_type_instructions(answer_type):
    """Get specific instructions based on answer type"""
    instructions = {
        "Informative": """
        - Provide comprehensive background information and facts
        - Include detailed explanations and context
        - Cover multiple aspects of the topic
        - Use evidence and examples to support points
        - Structure information clearly and logically""",

        "Practical Tips": """
        - Focus on actionable advice and guidance
        - Provide step-by-step instructions where applicable
        - Include specific recommendations and best practices
        - Emphasize what the user can actually do
        - Make suggestions concrete and implementable""",

        "Basic Overview": """
        - Keep explanations simple and accessible
        - Focus on key points and essential information
        - Avoid technical jargon or complex details
        - Provide a clear, easy-to-understand introduction
        - Structure information in a beginner-friendly way""",

        "Expert Analysis": """
        - Provide in-depth professional perspective
        - Include technical details and advanced insights
        - Reference industry standards and best practices
        - Demonstrate specialized knowledge and expertise
        - Address complex aspects and nuances""",

        "Comparison": """
        - Present pros and cons clearly
        - Compare different options or approaches
        - Provide balanced evaluation of alternatives
        - Include recommendations based on comparison
        - Help user understand trade-offs""",

        "Problem-Solving": """
        - Focus on solutions and troubleshooting
        - Address specific issues and challenges
        - Provide practical problem-solving approaches
        - Include preventive measures where applicable
        - Emphasize resolution strategies"""
    }

    return instructions.get(answer_type, instructions["Informative"])

In [ ]:
def stance_analysis_enhanced(query, ling_response, expert_response, user_response, topic, stance, answer_type):
    """Enhanced stance analysis that considers answer type"""
    role_1 = target_role_map.get("Local")
    role_2 = target_role_map.get("Expert")
    role_3 = target_role_map.get("User Analysis")

    stance_context = {
        "positive": "highly beneficial, well-founded, and strongly recommended",
        "negative": "problematic, risky, or not advisable"
    }

    stance_description = stance_context.get(stance, stance)
    answer_type_main = answer_type.split('(')[0].strip()

    prompt = f"""You are conducting stance detection analysis for collaborative decision-making.

    Original Query: '''{query}'''
    Topic: {topic}
    Requested Answer Type: {answer_type}

    EXPERT ANALYSES:
    From {role_1}: <<<{ling_response}>>>
    From {role_2}: [[[{expert_response}]]]
    From {role_3}: ---{user_response}---

    YOUR STANCE: You believe the approaches, recommendations, or solutions presented in response to this query are {stance_description} for the user's situation regarding {topic}.

    IMPORTANT: Your argument should be formatted as {answer_type_main.upper()} since that's what the user requested.

    {get_answer_type_instructions(answer_type_main)}

    TASK:
    1. **Analyze all three expert perspectives** through your {stance} lens
    2. **Extract supporting evidence** that supports your {stance} position
    3. **Build your argument** using evidence while following the {answer_type_main} format

    Present your {stance} argument with specific evidence from the expert analyses, formatted according to the user's requested answer type."""

    return get_completion(prompt)

In [ ]:
def final_judgement(query, favor_response, against_response, topic):
    """
    Enhanced final judgement that synthesizes collaborative analysis
    """
    prompt = f"""You are the final decision-maker in a collaborative analysis system. Your role is to synthesize multiple expert perspectives and opposing viewpoints to provide the best possible response to the user.

    USER QUERY: "{query}"
    TOPIC AREA: {topic}

    COLLABORATIVE ANALYSIS RESULTS:

    POSITIVE PERSPECTIVE (Supporting Arguments):
    {favor_response}

    NEGATIVE PERSPECTIVE (Cautionary Arguments):
    {against_response}

    YOUR TASK:
    Synthesize these collaborative analyses to provide the optimal response to the user's query. This means:

    1. **Evaluate evidence quality**: Assess the strength and credibility of arguments from both sides
    2. **Consider user context**: Focus on what would be most beneficial for someone asking this specific query
    3. **Balance perspectives**: Integrate the strongest insights from both positive and negative analyses
    4. **Provide actionable guidance**: Give the user clear, practical direction

    OUTPUT REQUIREMENTS:
    - Deliver a comprehensive yet concise response
    - Be definitive while acknowledging important considerations
    - Focus on practical value for the user
    - Integrate insights from the collaborative analysis
    - Present as the authoritative answer to their query
    - Give a brief, practical response (1-2 paragraphs).

    Your response should represent the best collective wisdom from the collaborative analysis process."""

    judgement = get_completion(prompt)
    return judgement


In [ ]:
def final_judgement_enhanced(query, favor_response, against_response, topic, answer_type):
    """Enhanced final judgement that considers answer type"""
    answer_type_main = answer_type.split('(')[0].strip()

    prompt = f"""You are the final decision-maker in a collaborative analysis system. Your role is to synthesize multiple expert perspectives and opposing viewpoints to provide the best possible response to the user.

    USER QUERY: "{query}"
    TOPIC AREA: {topic}
    REQUESTED ANSWER TYPE: {answer_type}

    COLLABORATIVE ANALYSIS RESULTS:

    POSITIVE PERSPECTIVE (Supporting Arguments):
    {favor_response}

    NEGATIVE PERSPECTIVE (Cautionary Arguments):
    {against_response}

    YOUR TASK:
    Synthesize these collaborative analyses to provide the optimal response to the user's query.

    CRITICAL: Format your response as {answer_type_main.upper()} as specifically requested by the user:

    {get_answer_type_instructions(answer_type_main)}

    OUTPUT REQUIREMENTS:
    - Follow the {answer_type_main} format strictly
    - Integrate insights from the collaborative analysis
    - Focus on practical value for the user
    - Be definitive while acknowledging important considerations
    - Present as the authoritative answer to their query

    Your response should represent the best collective wisdom from the collaborative analysis process, delivered in exactly the format the user requested."""

    judgement = get_completion(prompt)
    return judgement


In [ ]:
# Test your private LLM
def test_private_llm():
    response = client.chat.completions.create(
        model="llama-3.3-70b-instruct",
        messages=[
            {"role": "user", "content": "Explain how AI works in a few words"}
        ]
    )
    print("Private LLM Response:", response.choices[0].message.content)

test_private_llm()

Private LLM Response: Data + Algorithms = Insights


In [ ]:
search_tool = GoogleSearchAPIWrapper()

In [ ]:
# SIMPLE APPROACH: Use your private LLM directly with LangChain
# Since your server uses OpenAI syntax, you can use ChatOpenAI directly!
private_llm = ChatOpenAI(
    base_url="http://api.llm.apps.os.dcs.gla.ac.uk/v1",
    api_key=os.environ['IDA_LLM_API_KEY'],
    model="llama-3.3-70b-instruct",
    temperature=0
)

In [ ]:
# Create the agent with your private LLM
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful travel assistant. When you need current information, use the web_search tool."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

In [ ]:
tools = [
    Tool(
        name="web_search",
        func=search_tool.run,
        description="Fetches real-time information via web search.",
    )
]

In [ ]:
agent = create_tool_calling_agent(private_llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
def simple_rag_with_private_llm(query):
    """
    Improved RAG function with better search query extraction
    """
    print(f"RAG processing query: {query[:100]}...")

    try:
        # Extract key search terms from the complex query
        # Look for the original user query in the prompt
        if 'USER\'S QUERY:' in query:
            # Extract the actual user query for searching
            lines = query.split('\n')
            for line in lines:
                if 'USER\'S QUERY:' in line:
                    user_query = line.split('USER\'S QUERY:')[1].strip().strip('"')
                    search_query = user_query
                    break
        elif 'QUERY:' in query:
            lines = query.split('\n')
            for line in lines:
                if 'QUERY:' in line and not 'USER\'S' in line:
                    search_query = line.split('QUERY:')[1].strip().strip('"')
                    break
        else:
            # Fallback - use first 50 characters
            search_query = query[:50]

        print(f"Extracted search query: {search_query}")

        # Search with the extracted query
        search_results = search_tool.run(search_query)
        print(f"Search results: {search_results[:200]}...")

        # Check if search was successful
        if "No good Google Search Result was found" in search_results:
            print("No search results found, providing response based on original recommendation")
            return "Based on current information, the original expert recommendation remains valid. No significant updates or changes were found that would alter the core advice provided."

        # Enhanced RAG prompt for better synthesis
        rag_prompt = f"""Based on the search results, enhance and validate the expert recommendation.

        ORIGINAL EXPERT RECOMMENDATION: {query}

        CURRENT SEARCH RESULTS:
        {search_results}

        TASK: Use these search results to validate, update, and enhance the expert recommendation. Focus on:
        - Current accuracy of the information
        - Recent developments or changes
        - Specific details that improve the recommendation
        - Any corrections needed based on current data

        Provide a clear, enhanced recommendation."""

        response = client.chat.completions.create(
            model="llama-3.3-70b-instruct",
            messages=[
                {"role": "system", "content": "You are a research analyst who validates expert recommendations using current search results. Focus on practical improvements and current accuracy."},
                {"role": "user", "content": rag_prompt}
            ],
            temperature=0
        )

        return response.choices[0].message.content

    except Exception as e:
        print(f"RAG search error: {e}")
        return "Unable to retrieve current information for validation. The original expert recommendation stands as provided."

In [ ]:
# Agent-based RAG function
def agent_rag_with_private_llm(query):
    """
    Agent-based RAG: Let the agent decide when to search and how to respond
    """
    result = agent_executor.invoke({"input": query})
    return result["output"]

In [ ]:
#assign experts for target
def define_roles(definition_list):
    global target_role_map
    target_role_map = {
        "Local": definition_list[1],
        "Expert": definition_list[2],
        "User Analysis": definition_list[3]
    }

In [ ]:
# Enhanced COLA framework function that incorporates answer type
def add_predictions_sequential_enhanced(original_query, selected_intent, selected_answer_type, query_id):
    global topic
    start_time = time.time()

    print(f"Processing query {query_id}")
    print(f"Original query: {original_query}")
    print(f"Selected intent: {selected_intent}")
    print(f"Selected answer type: {selected_answer_type}")
    print("---------DUAL INTENT CLARIFICATION COMPLETE--------\n")

    # STEP 1: Rewrite query using both intents
    working_query = rewrite_query_with_dual_intent(original_query, selected_intent, selected_answer_type)
    print(f"Rewritten query: {working_query}")
    print("---------QUERY REWRITING COMPLETE--------\n")

    # STEP 2: Get roles and topic based on the rewritten query
    definition_list = get_roles(working_query)
    topic = definition_list[0]
    define_roles(definition_list)

    print(f"Identified topic: {topic}")
    print(f"Assigned roles: {definition_list[1:]}")
    print("---------ROLE ASSIGNMENT COMPLETE--------\n")

    # STEP 3: Enhanced analysis functions that consider answer type
    ling_response = local_analysis_enhanced(working_query, topic, selected_answer_type)
    print("---------LOCAL RESPONSE--------\n")
    print(ling_response)

    expert_response = expert_analysis_enhanced(working_query, topic, selected_answer_type)
    print("---------EXPERT RESPONSE--------\n")
    print(expert_response)

    user_response = user_analysis_enhanced(working_query, topic, selected_answer_type)
    print("---------USER ANALYSIS RESPONSE--------\n")
    print(user_response)

    # STEP 4: Enhanced stance analysis
    favor_response = stance_analysis_enhanced(working_query, ling_response, expert_response, user_response, topic, "positive", selected_answer_type)
    print("---------IN FAVOR RESPONSE--------\n")
    print(favor_response)

    against_response = stance_analysis_enhanced(working_query, ling_response, expert_response, user_response, topic, "negative", selected_answer_type)
    print("---------AGAINST RESPONSE--------\n")
    print(against_response)

    # STEP 5: Enhanced final judgement
    final_response = final_judgement_enhanced(working_query, favor_response, against_response, topic, selected_answer_type)
    print("---------ENHANCED COLA FRAMEWORK ANSWER--------\n")
    print(final_response)

    # Calculate processing time
    end_time = time.time()
    processing_time = round(end_time - start_time, 2)
    print(f"Total processing time: {processing_time} seconds")

    # Store results with all the information
    results = {
        'Rewritten_Query': working_query,  # Store the rewritten query
        'Selected_Topic_Intent': selected_intent,  # Store topic selection
        'Selected_Answer_Type': selected_answer_type,  # Store answer type selection
        'Linguist Analysis': ling_response,
        'Expert Analysis': expert_response,
        'User Analysis': user_response,
        'In Favor': favor_response,
        'Against': against_response,
        'Final Judgement': final_response,
        'After RAG Agent': '',
        'Final Plan': '',
        'Processing_Time_Seconds': processing_time
    }

    update_query_results(query_id, results)
    return str(final_response)

In [ ]:
def get_last_query_data():
    """
    Clean version - get the last query data from database
    """
    try:
        if not os.path.exists('cola_database.csv'):
            return None, None, None

        df = pd.read_csv('cola_database.csv')
        if df.empty:
            return None, None, None

        # Try completed queries first, fallback to any query
        completed_queries = df[df['Status'] == 'completed']
        if not completed_queries.empty:
            last_query = completed_queries.iloc[-1]
        else:
            last_query = df.iloc[-1]

        query_id = int(last_query['ID'])
        query = str(last_query['Query'])

        # Handle different possible column names
        if 'Final Judgement' in df.columns:
            final_response = str(last_query['Final Judgement'])
        elif 'Final_Judgement' in df.columns:
            final_response = str(last_query['Final_Judgement'])
        else:
            final_response = query

        return query_id, query, final_response

    except Exception as e:
        print(f"Error retrieving last query data: {e}")
        return None, None, None

In [ ]:
def final_response_rag(final_response, topic, query):
    """
    Enhanced RAG prompt - more focused and concise
    """
    response = f"""You are validating and enhancing an expert recommendation with current information.

    USER'S QUERY: "{query}"
    TOPIC: {topic}
    EXPERT RECOMMENDATION: {final_response}

    Your task:
    1. Verify the recommendation against current information
    2. Update any outdated details with recent data
    3. Add missing important information
    4. Enhance with specific, current details

    Provide a refined recommendation that:
    - Incorporates latest information
    - Addresses the user's specific query
    - Is practical and actionable
    - Explains any significant updates made

    Focus on improving the original recommendation, not creating a detailed plan."""

    return response

In [ ]:
def final_plan_rag(query_id, query, final_response):
    start_time = time.time()

    # Get topic from the database or global variable
    global topic  # Make sure topic is accessible

    print(f"Starting RAG refinement for query: {query}")
    print(f"Topic: {topic}")
    print(f"Original COLA response: {final_response[:200]}...")

    # RAG - Generate enhanced prompt and get current information
    prompt = final_response_rag(final_response, topic, query)
    response = simple_rag_with_private_llm(prompt)
    print("---------RAG ENHANCED RESPONSE--------\n")
    print(response)

    print("---------FINAL RAG REFINED ANSWER--------\n")
    print(response)

    # Calculate processing time
    end_time = time.time()
    processing_time = round(end_time - start_time, 2)
    print(f"Total processing time: {processing_time} seconds")
    # Debug: Check what we actually received
    print(f"DEBUG - query_id type: {type(query_id)}, value: {query_id}")
    print(f"DEBUG - query type: {type(query)}, value: {query}")

    # Try to update database, but don't fail if it doesn't work
    try:
        results = {
            'After RAG Agent': str(response),
            'Final Plan': str(response),
            'Processing_Time_Seconds_RAG': processing_time,
        }
        update_query_results(query_id, results)
        print(f"Database updated successfully for query ID {query_id}")
    except Exception as e:
        print(f"Database update failed for query ID {query_id}: {e}")
        print("RAG completed successfully despite database update failure")

    return str(response)  # Return the RAG-enhanced response regardless of database status

In [ ]:
def execute_rag_update(chatbot_history, state):
    """
    Debug version to see what's happening
    """
    try:
        print("DEBUG - execute_rag_update started")

        # Get the last query data
        query_id, query, final_response = get_last_query_data()
        print(f"DEBUG - get_last_query_data returned: {query_id}, {query}, {final_response}")

        if not query_id:
            print("DEBUG - No query_id found, returning error")
            error_msg = "❌ No completed queries found in database. Please submit a query first."
            new_message = {"role": "assistant", "content": error_msg}
            updated_history = chatbot_history + [new_message]
            updated_state = state + [new_message]
            return updated_history, updated_state, ""

        # Show processing message
        processing_msg = f"🔄 **Enhancing answer with current information...**\n\nOriginal query: '{query[:100]}...'"
        processing_message = {"role": "assistant", "content": processing_msg}
        temp_history = chatbot_history + [processing_message]

        yield temp_history, state, ""

        print("DEBUG - About to call final_plan_rag")
        # Call RAG function
        enhanced_response = final_plan_rag(query_id, query, final_response)
        print("DEBUG - final_plan_rag completed")

        # Show enhanced result
        success_msg = f"✅ **Answer enhanced with current information!**\n\n{enhanced_response}"
        final_message = {"role": "assistant", "content": success_msg}

        updated_history = chatbot_history + [final_message]
        updated_state = state + [final_message]

        yield updated_history, updated_state, ""

    except Exception as e:
        print(f"DEBUG - Exception in execute_rag_update: {e}")
        import traceback
        traceback.print_exc()
        error_msg = f"❌ **Error enhancing answer:** {str(e)}"
        error_message = {"role": "assistant", "content": error_msg}
        updated_history = chatbot_history + [error_message]
        updated_state = state + [error_message]
        yield updated_history, updated_state, ""

In [ ]:
def slow_echo_with_dual_intent_disambiguation(message, history):
    global j
    j += 1
    current_id = j

    # Add user message to history
    history.append({"role": "user", "content": message})
    yield history, history, ""

    try:
        # Step 1: Add query to database
        add_new_query(current_id, message)

        # Step 2: Generate topic intent options
        topic_intent_options = generate_intent_options(message)
        print(f"Original: {message}")
        print(f"Topic intent options: {topic_intent_options}")

        # Step 3: Show topic options to user
        updated_history, updated_state = show_intent_options(history, history, topic_intent_options)
        yield updated_history, updated_state, ""

        # Store the options and query_id globally for button handlers
        global current_intent_options, current_query_id, current_original_query, current_step
        current_intent_options = topic_intent_options
        current_query_id = current_id
        current_original_query = message
        current_step = "topic_selection"  # Track which step we're on

    except Exception as e:
        error_message = f"Error in intent disambiguation: {str(e)}"
        print(f"Error: {e}")
        history[-1] = {"role": "assistant", "content": error_message}
        yield history, history, ""

In [ ]:
# Global variables to store current session data
current_intent_options = []
current_query_id = None
current_original_query = ""
current_step = "topic_selection"  # Can be "topic_selection" or "answer_type_selection"
current_answer_type_options = []
current_selected_topic_intent = ""

In [ ]:
def on_button_click():
    return "Button clicked!"



In [ ]:
def view_database_stats():
    """View database statistics"""
    try:
        df = pd.read_csv('cola_database.csv', dtype={
            'ID': 'int64',
            'Query': 'string',
            'Linguist Analysis': 'string',
            'Expert Analysis': 'string',
            'User Analysis': 'string',
            'In Favor': 'string',
            'Against': 'string',
            'Final Judgement': 'string',
            'After RAG Agent': 'string',
            'Final Plan': 'string',
            'Processing_Time_Seconds': 'float64',
            'Processing_Time_Seconds_RAG': 'float64',
            'Timestamp': 'string',
            'Status': 'string'
        })

        total_queries = len(df)
        completed = len(df[df['Status'] == 'completed'])
        pending = len(df[df['Status'] == 'pending'])
        errors = len(df[df['Status'] == 'error'])

        # Calculate average processing time for completed queries
        completed_df = df[df['Status'] == 'completed']
        if not completed_df.empty and 'Processing_Time_Seconds' in completed_df.columns:
            # Filter out NaN values before calculating mean
            processing_times = completed_df['Processing_Time_Seconds'].dropna()
            if not processing_times.empty:
                avg_time = processing_times.mean()
                avg_time_str = f"- Average processing time: {avg_time:.2f} seconds"
            else:
                avg_time_str = "- Average processing time: N/A"
        else:
            avg_time_str = "- Average processing time: N/A"

        stats = f"""
        Database Statistics:
        - Total queries: {total_queries}
        - Completed: {completed}
        - Pending: {pending}
        - Errors: {errors}
        {avg_time_str}

        Recent queries:
        """

        if not df.empty:
            recent = df.tail(5)[['ID', 'Query', 'Status', 'Processing_Time_Seconds', 'Timestamp']]
            stats += recent.to_string(index=False)

        return stats
    except Exception as e:
        return f"Error reading database: {e}"

In [ ]:
def handle_option_1_click_enhanced(chatbot_history, state):
    global current_step, current_intent_options, current_answer_type_options, current_selected_topic_intent

    if current_step == "topic_selection" and current_intent_options and len(current_intent_options) > 0:
        # First step: topic selection
        selected_topic = current_intent_options[0]
        current_selected_topic_intent = selected_topic

        # Generate answer type options
        answer_type_options = generate_answer_type_options(current_original_query, selected_topic)
        current_answer_type_options = answer_type_options
        current_step = "answer_type_selection"

        # Remove topic selection message and show answer type options
        cleaned_history = chatbot_history[:-1] if chatbot_history else []
        cleaned_state = state[:-1] if state else []

        updated_history, updated_state = show_answer_type_options(cleaned_history, cleaned_state, answer_type_options)
        yield updated_history, updated_state, ""

    elif current_step == "answer_type_selection" and current_answer_type_options and len(current_answer_type_options) > 0:
        # Second step: answer type selection
        selected_answer_type = current_answer_type_options[0]

        # Remove answer type selection message and process
        cleaned_history = chatbot_history[:-1] if chatbot_history else []
        cleaned_state = state[:-1] if state else []

        yield from process_selected_answer_type(
            selected_answer_type,
            current_selected_topic_intent,
            current_original_query,
            current_query_id,
            cleaned_history,
            cleaned_state
        )

        # Reset step
        current_step = "topic_selection"
    else:
        yield chatbot_history, state, ""

def handle_option_2_click_enhanced(chatbot_history, state):
    global current_step, current_intent_options, current_answer_type_options, current_selected_topic_intent

    if current_step == "topic_selection" and current_intent_options and len(current_intent_options) > 1:
        selected_topic = current_intent_options[1]
        current_selected_topic_intent = selected_topic

        answer_type_options = generate_answer_type_options(current_original_query, selected_topic)
        current_answer_type_options = answer_type_options
        current_step = "answer_type_selection"

        cleaned_history = chatbot_history[:-1] if chatbot_history else []
        cleaned_state = state[:-1] if state else []

        updated_history, updated_state = show_answer_type_options(cleaned_history, cleaned_state, answer_type_options)
        yield updated_history, updated_state, ""

    elif current_step == "answer_type_selection" and current_answer_type_options and len(current_answer_type_options) > 1:
        selected_answer_type = current_answer_type_options[1]

        cleaned_history = chatbot_history[:-1] if chatbot_history else []
        cleaned_state = state[:-1] if state else []

        yield from process_selected_answer_type(
            selected_answer_type,
            current_selected_topic_intent,
            current_original_query,
            current_query_id,
            cleaned_history,
            cleaned_state
        )

        current_step = "topic_selection"
    else:
        yield chatbot_history, state, ""

def handle_option_3_click_enhanced(chatbot_history, state):
    global current_step, current_intent_options, current_answer_type_options, current_selected_topic_intent

    if current_step == "topic_selection" and current_intent_options and len(current_intent_options) > 2:
        selected_topic = current_intent_options[2]
        current_selected_topic_intent = selected_topic

        answer_type_options = generate_answer_type_options(current_original_query, selected_topic)
        current_answer_type_options = answer_type_options
        current_step = "answer_type_selection"

        cleaned_history = chatbot_history[:-1] if chatbot_history else []
        cleaned_state = state[:-1] if state else []

        updated_history, updated_state = show_answer_type_options(cleaned_history, cleaned_state, answer_type_options)
        yield updated_history, updated_state, ""

    elif current_step == "answer_type_selection" and current_answer_type_options and len(current_answer_type_options) > 2:
        selected_answer_type = current_answer_type_options[2]

        cleaned_history = chatbot_history[:-1] if chatbot_history else []
        cleaned_state = state[:-1] if state else []

        yield from process_selected_answer_type(
            selected_answer_type,
            current_selected_topic_intent,
            current_original_query,
            current_query_id,
            cleaned_history,
            cleaned_state
        )

        current_step = "topic_selection"
    else:
        yield chatbot_history, state, ""

In [ ]:
def create_enhanced_gradio_interface():
    with gr.Blocks() as demo:
        # Initialize database on startup
        initialize_database()

        chatbot = gr.Chatbot(
            label="Enhanced Collaborative Search with Dual Intent Clarification",
            type="messages",
            height=400
        )
        msg = gr.Textbox(label="Your query", placeholder="How can I help you today?")
        send_btn = gr.Button("Send")

        # Intent selection buttons (now handle both topic and answer type selection)
        with gr.Row():
            option1_btn = gr.Button("Option 1", size="lg", variant="secondary")
            option2_btn = gr.Button("Option 2", size="lg", variant="secondary")
            option3_btn = gr.Button("Option 3", size="lg", variant="secondary")

        extra_btn = gr.Button("Update information using RAG")

        # Database management buttons
        with gr.Row():
            stats_btn = gr.Button("View Database Stats")
            export_btn = gr.Button("Export Database")

        stats_output = gr.Textbox(label="Database Information", lines=10)
        state = gr.State([])

        # Event handlers - REPLACE your existing handlers with these
        send_btn.click(
            slow_echo_with_dual_intent_disambiguation,
            [msg, state],
            [chatbot, state, msg]
        )
        msg.submit(
            slow_echo_with_dual_intent_disambiguation,
            [msg, state],
            [chatbot, state, msg]
        )

        # Enhanced intent option handlers - REPLACE your existing button handlers
        option1_btn.click(
            handle_option_1_click_enhanced,
            [chatbot, state],
            [chatbot, state, msg]
        )
        option2_btn.click(
            handle_option_2_click_enhanced,
            [chatbot, state],
            [chatbot, state, msg]
        )
        option3_btn.click(
            handle_option_3_click_enhanced,
            [chatbot, state],
            [chatbot, state, msg]
        )

        # Keep your existing handlers
        extra_btn.click(execute_rag_update, [chatbot, state], [chatbot, state, msg])
        stats_btn.click(view_database_stats, outputs=stats_output)

        # Keep your existing launcher
        demo.launch(share=True)

In [ ]:
create_enhanced_gradio_interface()

Enhanced COLA database initialized
* Running on local URL:  http://127.0.0.1:7864
* Running on public URL: https://3da7a7b5d897f66065.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Added query 1 to enhanced COLA database
Original: I want to know more about Panama
Topic intent options: ['Panama (country in Central America)', 'Panama (hat style)', 'Panama (canal and transportation hub)']
Processing query 1
Original query: I want to know more about Panama
Selected intent: Panama (country in Central America)
Selected answer type: Basic Overview (simple introduction for beginners)
---------DUAL INTENT CLARIFICATION COMPLETE--------

Rewritten query: Provide a basic overview and introduction to Panama, a country in Central America, covering its key features, geography, culture, and essential information for beginners.
---------QUERY REWRITING COMPLETE--------

Identified topic: [
Assigned roles: "Introduction to Panama", "Geographer", "Cultural Anthropologist", "Travel Consultant"]
---------ROLE ASSIGNMENT COMPLETE--------

---------LOCAL RESPONSE--------

BASIC OVERVIEW:

Panama is a country located in Central America, connecting North America to South America. Here a